In [ ]:
import pandas as pd
import re
import numpy as np
import datetime as dt

In [ ]:
from google.colab import files 
uploaded = files.upload()

# please upload customer_search.csv, ItemSoldoveraperiodDigicorResults.xls and bom_result.csv

Saving ItemSoldoveraperiodDigicorResults.xls to ItemSoldoveraperiodDigicorResults.xls
Saving customer_search.csv to customer_search.csv
Saving bom_result.csv to bom_result.csv


# Import Data & Aggregate Data By Month

In [ ]:
#Convert to dataframe
fields = ['name', 'desc', 'Category', 'bom_type ID', 'BOM Type'] #Remove unwanted columns
data_byMonth = pd.read_csv('customer_search.csv') #Jul19-Sep21
qtySold_byMonth = pd.read_excel('ItemSoldoveraperiodDigicorResults.xls') #Aug18-Aug21
bom_result = pd.read_csv('bom_result.csv', usecols = fields)

In [ ]:
# Avg. Time on Page: Null values are interpreted as the user clicking on page and not having any activity and or 
# has immediately left the page, therefore null values are changed to 0
data_byMonth['Avg. Time on Page'] = data_byMonth['Avg. Time on Page'].fillna(0)
data_byMonth.isnull().sum()

Date                           0
Page                           0
Page Title                     0
Pageviews                      0
Organic Searches               0
New Users                      0
Avg. Time on Page              0
Number of Sessions per User    0
Users                          0
dtype: int64

In [ ]:
# Remove items that aren't 'BB-'
qtySold_byMonth = qtySold_byMonth[qtySold_byMonth['Item Code'].str.contains("BB-")]

In [ ]:
#Strip 'BB-': So we can match with other tables
qtySold_byMonth['Item Code'] = qtySold_byMonth['Item Code'].map(lambda x: x.lstrip('BB-'))

In [ ]:
qtySold_byMonth

,Item Code,Doc Date,Qty Sold
57,1013S-MTR,2018-11-29,1.0
58,1013S-MTR,2018-09-25,1.0
59,1013S-MTR,2018-09-24,2.0
60,1013S-MTR,2020-06-16,2.0
61,1013S-MTR,2020-06-16,2.0
...,...,...,...
692,SBI-7428R-C3N,2019-05-30,2.0
693,SBI-7428R-C3N,2019-10-25,8.0
694,SBI-7428R-C3N,2019-10-25,6.0
695,SBI-7428R-C3N,2019-06-17,11.0


In [ ]:
qtySold_byMonth['Doc Date'] = pd.to_datetime(qtySold_byMonth["Doc Date"].dt.strftime('%Y-%m'))

In [ ]:
# Aggregate qty sold, group by datetime: Month
qtySold_byMonth = qtySold_byMonth.groupby(by=[qtySold_byMonth['Doc Date'], qtySold_byMonth['Item Code']]).agg({'Qty Sold': [sum]})

In [ ]:
# Groupby creates multi index df, must reset
qtySold_byMonth.columns = qtySold_byMonth.columns.droplevel(level=0)
qtySold_byMonth = qtySold_byMonth.add_suffix('_sum').reset_index()

In [ ]:
qtySold_byMonth = qtySold_byMonth.rename(columns={"sum_sum": "Qty Sold"})

In [ ]:
qtySold_byMonth

,Doc Date,Item Code,Qty Sold
0,2018-08-01,5018A-FTN4,4.0
1,2018-08-01,5018D-FN8T,6.0
2,2018-08-01,5018R-M,3.0
3,2018-08-01,5019S-L,2.0
4,2018-08-01,5019S-ML,8.0
...,...,...,...
488,2021-08-01,5014A-TT,1.0
489,2021-08-01,5019A-FTN4,1.0
490,2021-08-01,5019P-MT,1.0
491,2021-08-01,6019P-WTR,3.0


In [ ]:
data_byMonth['Date'] = pd.to_datetime(data_byMonth['Date'], format="%b %d, %Y")

In [ ]:
data_byMonth['Date'] = pd.to_datetime(data_byMonth["Date"].dt.strftime('%Y-%m'))

In [ ]:
avgt_pv = data_byMonth['Avg. Time on Page']*data_byMonth['Pageviews']

In [ ]:
data_byMonth['avg*pv'] = avgt_pv

In [ ]:
x_byMonth = data_byMonth.groupby(by=[data_byMonth['Date'], data_byMonth['Page']]).agg({
    
    'Pageviews' : [sum],
    
    'Organic Searches': [sum],
    
    'New Users': [sum],
    
    'Avg. Time on Page': [sum],

    'Number of Sessions per User': [sum],

    'Users': [sum],

    'avg*pv': [sum]
  }  
)

In [ ]:
x_byMonth.columns = x_byMonth.columns.droplevel(level=0)
x_byMonth = x_byMonth.add_suffix('_sum').reset_index()

In [ ]:
x_byMonth = x_byMonth.set_axis(['Date', 'Page', 'Page Views', 'Organic Searches', 'New Users', 'Avg. Time on Page', 'Number of Sessions per User', 'Users', 'avg*sum'], axis=1, inplace=False)

In [ ]:
weighted_avg = x_byMonth['avg*sum']/x_byMonth['Page Views']

In [ ]:
x_byMonth['Weighted Avg Time on Page'] = weighted_avg

In [ ]:
data_byMonth = x_byMonth

In [ ]:
data_byMonth

,Date,Page,Page Views,Organic Searches,New Users,Avg. Time on Page,Number of Sessions per User,Users,avg*sum,Weighted Avg Time on Page
0,2019-07-01,/,1024,73,242,1619.111250,13.189989,403,161613.21608,157.825406
1,2019-07-01,/5-tangible-benefits-digital-transformation/,69,58,61,284.000000,9.111111,65,3259.00000,47.231884
2,2019-07-01,/?source=post_page---------------------------,1,0,1,0.000000,1.000000,1,0.00000,0.000000
3,2019-07-01,/DiGiCOR High Density Server/DiGiCOR-High-Dens...,1,0,0,28.000000,0.000000,1,28.00000,28.000000
4,2019-07-01,/Intel/intel-server,3,0,0,94.666667,0.000000,1,284.00000,94.666667
...,...,...,...,...,...,...,...,...,...,...
13078,2021-09-01,/systems?mkt_tok=Mjk0LVlRSy03OTcAAAF_j_tkxFE57...,1,0,1,0.000000,1.000000,1,0.00000,0.000000
13079,2021-09-01,/systems?mkt_tok=Mjk0LVlRSy03OTcAAAF_j_tkxFahn...,1,0,1,0.000000,1.000000,1,0.00000,0.000000
13080,2021-09-01,/systems?mkt_tok=Mjk0LVlRSy03OTcAAAF_j_tkxLj4x...,1,0,0,0.000000,0.000000,1,0.00000,0.000000
13081,2021-09-01,/website-policy,1,0,1,64.000000,1.000000,1,64.00000,64.000000


In [ ]:
qtySold_byMonth['Page'] = "n/a"

In [ ]:
#For each item in qtySold, match with the page in data using regex
for item in qtySold_byMonth['Item Code']:
  for page in data_byMonth['Page']:
    x = re.search(item + '$', page)
    if x != None:
      qtySold_byMonth.loc[qtySold_byMonth['Item Code'] == item, 'Page'] = page

In [ ]:
# Rename column for merging
bom_result = bom_result.rename(columns={"desc": "Item Code"})

In [ ]:
# Merge bom_result and qtySold
qtySold_bom_result = pd.merge(qtySold_byMonth, bom_result, how = "inner", on = ['Item Code'])

In [ ]:
# No need for rows with NaN quantity sold. Removed.
#qtySold_bom_result = qtySold_bom_result.dropna(subset=['Date'])

# Some names not specified, replaced with 'n/a'
qtySold_bom_result['name'].fillna("n/a", inplace = True)

In [ ]:
# bom_type ID and BOM type is what we need for category details
# Ignore date, just interested in qty sold
qtySold_bom_result = qtySold_bom_result.drop(columns=['name', 'Category'])

In [ ]:
qtySold_bom_result = qtySold_bom_result.rename(columns={"Doc Date": "Date"})
qtySold_bom_result = qtySold_bom_result.drop_duplicates() # Somehow there was a lot of duplicates?

qtySold_bom_result

,Date,Item Code,Qty Sold,Page,bom_type ID,BOM Type
0,2018-08-01,5018D-FN8T,6.0,/systems/SuperServer-5018D-FN8T,2,BB
1,2020-06-01,5018D-FN8T,1.0,/systems/SuperServer-5018D-FN8T,2,BB
2,2020-09-01,5018D-FN8T,5.0,/systems/SuperServer-5018D-FN8T,2,BB
3,2020-10-01,5018D-FN8T,5.0,/systems/SuperServer-5018D-FN8T,2,BB
4,2020-11-01,5018D-FN8T,3.0,/systems/SuperServer-5018D-FN8T,2,BB
...,...,...,...,...,...,...
621,2021-05-01,6029P-TRT,1.0,/systems/SuperServer-6029P-TRT,2,BB
622,2021-06-01,2029U-E1CRT,2.0,/systems/SuperServer-2029U-E1CRT,2,BB
623,2021-06-01,2113S-WN24RT,1.0,/systems/AS-2113S-WN24RT,2,BB
624,2021-07-01,6029P-WTR,1.0,/systems/SuperServer-6029P-WTR,2,BB


In [ ]:
### THIS WAS USED TO DOWNLOAD THE TABLE TO INSPECT IN EXCEL

# qtySold_bom_result.to_csv("MonthlyQtySold.csv", index = False)
# data_byMonth.to_csv("MonthlyWebsiteMetrics.csv", index = False)

In [ ]:
data_byMonth = data_byMonth.drop(['Avg. Time on Page', 'avg*sum'], axis = 1)

In [ ]:
print('qty sold earliest date:', qtySold_bom_result['Date'].min())
print('qty sold latest date:', qtySold_bom_result['Date'].max())
print('website metrics earliest date:', data_byMonth['Date'].min())
print('website metrics latest date:', data_byMonth['Date'].max())

qty sold earliest date: 2018-08-01 00:00:00
qty sold latest date: 2021-08-01 00:00:00
website metrics earliest date: 2019-07-01 00:00:00
website metrics latest date: 2021-09-01 00:00:00


# Complete Dataset

Based on time stamp above, everything seems correct. Now to create final dataset.

**Step 1**: Create a new table `key` two columns with date ranging from Jul19 - Aug21 (common time period between two datasets, based on timestamp printed above)

**Step 2:** Left Join `key` - `qtySold_bom_result` by *Date* and *Product Code* = `key2`

**Step 3:** Left Join `key2` - `data_byMonth` by *Date* and *Page* = `Final dataset`

First, we will filter our `qtySold_bom_result` and `data_byMonth` to the appropriate time period (i.e. Jul 2019 to Aug 2021)

In [ ]:
qtySold_bom_result = qtySold_bom_result[(qtySold_bom_result['Date'] > '2019-06-30') & (qtySold_bom_result['Date'] < '2021-08-31')]
qtySold_bom_result

,Date,Item Code,Qty Sold,Page,bom_type ID,BOM Type
1,2020-06-01,5018D-FN8T,1.0,/systems/SuperServer-5018D-FN8T,2,BB
2,2020-09-01,5018D-FN8T,5.0,/systems/SuperServer-5018D-FN8T,2,BB
3,2020-10-01,5018D-FN8T,5.0,/systems/SuperServer-5018D-FN8T,2,BB
4,2020-11-01,5018D-FN8T,3.0,/systems/SuperServer-5018D-FN8T,2,BB
15,2019-09-01,5019S-L,1.0,/systems/SuperServer-5019S-L,2,BB
...,...,...,...,...,...,...
621,2021-05-01,6029P-TRT,1.0,/systems/SuperServer-6029P-TRT,2,BB
622,2021-06-01,2029U-E1CRT,2.0,/systems/SuperServer-2029U-E1CRT,2,BB
623,2021-06-01,2113S-WN24RT,1.0,/systems/AS-2113S-WN24RT,2,BB
624,2021-07-01,6029P-WTR,1.0,/systems/SuperServer-6029P-WTR,2,BB


In [ ]:
data_byMonth = data_byMonth[(data_byMonth['Date'] > '2019-06-30') & (data_byMonth['Date'] < '2021-08-31')]
data_byMonth

,Date,Page,Page Views,Organic Searches,New Users,Number of Sessions per User,Users,Weighted Avg Time on Page
0,2019-07-01,/,1024,73,242,13.189989,403,157.825406
1,2019-07-01,/5-tangible-benefits-digital-transformation/,69,58,61,9.111111,65,47.231884
2,2019-07-01,/?source=post_page---------------------------,1,0,1,1.000000,1,0.000000
3,2019-07-01,/DiGiCOR High Density Server/DiGiCOR-High-Dens...,1,0,0,0.000000,1,28.000000
4,2019-07-01,/Intel/intel-server,3,0,0,0.000000,1,94.666667
...,...,...,...,...,...,...,...,...
12165,2021-08-01,/systems?AdvancedSearch=&Brand=INTEL,2,0,0,0.000000,1,23.000000
12166,2021-08-01,/systems?mkt_tok=Mjk0LVlRSy03OTcAAAF-6-CWc_dkO...,1,0,0,1.000000,1,0.000000
12167,2021-08-01,/userfiles/file/Digicor Profile.pdf,2,0,1,1.000000,1,25.500000
12168,2021-08-01,/website-policy,13,0,8,3.666667,11,98.230769


`x` is the list of dates we are going to use.

In [ ]:
daterange = pd.date_range('2019-07-1','2021-08-01', freq='MS').strftime("%Y-%m-%d %H:%M:%S").tolist()

dates_list = []
for i in range(len(daterange)):
  x = dt.datetime.strptime(daterange[i],'%Y-%m-%d %H:%M:%S')
  dates_list.append(x)

dates = pd.DataFrame(dates_list)
dates

,0
0,2019-07-01
1,2019-08-01
2,2019-09-01
3,2019-10-01
4,2019-11-01
5,2019-12-01
6,2020-01-01
7,2020-02-01
8,2020-03-01
9,2020-04-01


Now to extract the unique product codes.

In [ ]:
# itemcode = qtySold_bom_result['Item Code'].unique

itemcode = qtySold_bom_result['Item Code'].unique()
itemcode = pd.DataFrame(itemcode)

itemcode

,0
0,5018D-FN8T
1,5019S-L
2,5029A-2TN4
3,E200-9A
4,1013S-MTR
...,...
74,6029P-TRT
75,2029U-E1CRT
76,2113S-WN24RT
77,6029P-WTR


In [ ]:
# Create key to join dates and item code tables

dates['key'] = 1
itemcode['key'] = 1

# Merge both tables and drop key column

key = pd.merge(dates, itemcode, on ='key').drop("key", 1)

key

,0_x,0_y
0,2019-07-01,5018D-FN8T
1,2019-07-01,5019S-L
2,2019-07-01,5029A-2TN4
3,2019-07-01,E200-9A
4,2019-07-01,1013S-MTR
...,...,...
2049,2021-08-01,6029P-TRT
2050,2021-08-01,2029U-E1CRT
2051,2021-08-01,2113S-WN24RT
2052,2021-08-01,6029P-WTR


In [ ]:
# Rename column header

key = key.rename(columns = {'0_x': 'Date',
                      '0_y': 'Item Code'})

key

,Date,Item Code
0,2019-07-01,5018D-FN8T
1,2019-07-01,5019S-L
2,2019-07-01,5029A-2TN4
3,2019-07-01,E200-9A
4,2019-07-01,1013S-MTR
...,...,...
2049,2021-08-01,6029P-TRT
2050,2021-08-01,2029U-E1CRT
2051,2021-08-01,2113S-WN24RT
2052,2021-08-01,6029P-WTR


First, to join `key` with `qty sold` table

In [ ]:
firstjoin = pd.merge(key, qtySold_bom_result, how = 'left', on= ['Date', 'Item Code'])

The *Page*, *bom_type ID* and *BOM Type* column has not been joined properly, we will drop it and add it back in now that we have the correct number of rows in the table.

In [ ]:
firstjoin = firstjoin.drop(labels = ['Page', 'bom_type ID', 'BOM Type'], axis = 1)
firstjoin

,Date,Item Code,Qty Sold
0,2019-07-01,5018D-FN8T,NaN
1,2019-07-01,5019S-L,NaN
2,2019-07-01,5029A-2TN4,NaN
3,2019-07-01,E200-9A,NaN
4,2019-07-01,1013S-MTR,NaN
...,...,...,...
2049,2021-08-01,6029P-TRT,NaN
2050,2021-08-01,2029U-E1CRT,NaN
2051,2021-08-01,2113S-WN24RT,NaN
2052,2021-08-01,6029P-WTR,NaN


`item_info` below are the unique information for every *Item Code*, which will merged (left join) with `firstjoin`

In [ ]:
item_info = qtySold_bom_result.filter(['Item Code', 'Page', 'bom_type ID', 'BOM Type']).drop_duplicates()
item_info

,Item Code,Page,bom_type ID,BOM Type
1,5018D-FN8T,/systems/SuperServer-5018D-FN8T,2,BB
15,5019S-L,/systems/SuperServer-5019S-L,2,BB
27,5029A-2TN4,/systems/SuperServer-5029A-2TN4,2,BB
43,E200-9A,/systems/SuperServer-E200-9A,2,BB
48,1013S-MTR,/systems/AS-1013S-MTR,2,BB
...,...,...,...,...
621,6029P-TRT,/systems/SuperServer-6029P-TRT,2,BB
622,2029U-E1CRT,/systems/SuperServer-2029U-E1CRT,2,BB
623,2113S-WN24RT,/systems/AS-2113S-WN24RT,2,BB
624,6029P-WTR,/systems/SuperServer-6029P-WTR,2,BB


In [ ]:
firstjoin = pd.merge(firstjoin, item_info, how = 'left', on= ['Item Code'])
firstjoin

,Date,Item Code,Qty Sold,Page,bom_type ID,BOM Type
0,2019-07-01,5018D-FN8T,NaN,/systems/SuperServer-5018D-FN8T,2,BB
1,2019-07-01,5019S-L,NaN,/systems/SuperServer-5019S-L,2,BB
2,2019-07-01,5029A-2TN4,NaN,/systems/SuperServer-5029A-2TN4,2,BB
3,2019-07-01,E200-9A,NaN,/systems/SuperServer-E200-9A,2,BB
4,2019-07-01,1013S-MTR,NaN,/systems/AS-1013S-MTR,2,BB
...,...,...,...,...,...,...
2049,2021-08-01,6029P-TRT,NaN,/systems/SuperServer-6029P-TRT,2,BB
2050,2021-08-01,2029U-E1CRT,NaN,/systems/SuperServer-2029U-E1CRT,2,BB
2051,2021-08-01,2113S-WN24RT,NaN,/systems/AS-2113S-WN24RT,2,BB
2052,2021-08-01,6029P-WTR,NaN,/systems/SuperServer-6029P-WTR,2,BB


Now to add the monthly website metrics to get the final dataset

In [ ]:
joined_data = pd.merge(firstjoin, data_byMonth, how = 'left', on= ['Date', 'Page']).sort_values(['Item Code', 'Date'], ignore_index = True)
# joined_data = joined_data.reset_index(drop=True)

joined_data # the shape of this table should be 2054 x 14

,Date,Item Code,Qty Sold,Page,bom_type ID,BOM Type,Page Views,Organic Searches,New Users,Number of Sessions per User,Users,Weighted Avg Time on Page
0,2019-07-01,1013S-MTR,NaN,/systems/AS-1013S-MTR,2,BB,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-08-01,1013S-MTR,NaN,/systems/AS-1013S-MTR,2,BB,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-09-01,1013S-MTR,NaN,/systems/AS-1013S-MTR,2,BB,1.0,1.0,1.0,1.0,1.0,0.0
3,2019-10-01,1013S-MTR,NaN,/systems/AS-1013S-MTR,2,BB,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-11-01,1013S-MTR,NaN,/systems/AS-1013S-MTR,2,BB,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2049,2021-04-01,SBI-4129P-C2N,NaN,/systems/SBI-4129P-C2N,5,Node,NaN,NaN,NaN,NaN,NaN,NaN
2050,2021-05-01,SBI-4129P-C2N,NaN,/systems/SBI-4129P-C2N,5,Node,NaN,NaN,NaN,NaN,NaN,NaN
2051,2021-06-01,SBI-4129P-C2N,NaN,/systems/SBI-4129P-C2N,5,Node,NaN,NaN,NaN,NaN,NaN,NaN
2052,2021-07-01,SBI-4129P-C2N,NaN,/systems/SBI-4129P-C2N,5,Node,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
joined_data.isnull().sum()

Date                              0
Item Code                         0
Qty Sold                       1768
Page                              0
bom_type ID                       0
BOM Type                          0
Page Views                     1055
Organic Searches               1055
New Users                      1055
Number of Sessions per User    1055
Users                          1055
Weighted Avg Time on Page      1055
dtype: int64

In [ ]:
joined_data = joined_data.fillna(value = 0)
joined_data

,Date,Item Code,Qty Sold,Page,bom_type ID,BOM Type,Page Views,Organic Searches,New Users,Number of Sessions per User,Users,Weighted Avg Time on Page
0,2019-07-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-08-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-09-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,1.0,1.0,1.0,1.0,1.0,0.0
3,2019-10-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-11-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2049,2021-04-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.0
2050,2021-05-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.0
2051,2021-06-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.0
2052,2021-07-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
joined_data.isnull().sum() # No more missing values

Date                           0
Item Code                      0
Qty Sold                       0
Page                           0
bom_type ID                    0
BOM Type                       0
Page Views                     0
Organic Searches               0
New Users                      0
Number of Sessions per User    0
Users                          0
Weighted Avg Time on Page      0
dtype: int64

In [ ]:
# joined_data.to_csv("webdata_qtySold_byMonth.csv")

# Compute New Columns - Previous Months Web Metrics

**Step 1:** Subset the website metrics from final_data

**Step 2:** Use `pd.shift` function to compute values from previous row twice for *t-1* and *t-2*, grouped by `Item Code`

**Step 3:** Join the tables by index

In [ ]:
joined_data.columns

Index(['Date', 'Item Code', 'Qty Sold', 'Page', 'bom_type ID', 'BOM Type',
       'Page Views', 'Organic Searches', 'New Users',
       'Number of Sessions per User', 'Users', 'Weighted Avg Time on Page'],
      dtype='object')

In [ ]:
webmetrics = joined_data.filter(['Item Code', 'Page Views', 'Organic Searches', 'New Users', 'Avg. Time on Page', 'Number of Sessions per User', 'Users', 'avg*sum', 'Weighted Avg Time on Page'])

webmetrics

,Item Code,Page Views,Organic Searches,New Users,Number of Sessions per User,Users,Weighted Avg Time on Page
0,1013S-MTR,0.0,0.0,0.0,0.0,0.0,0.0
1,1013S-MTR,0.0,0.0,0.0,0.0,0.0,0.0
2,1013S-MTR,1.0,1.0,1.0,1.0,1.0,0.0
3,1013S-MTR,0.0,0.0,0.0,0.0,0.0,0.0
4,1013S-MTR,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...
2049,SBI-4129P-C2N,0.0,0.0,0.0,0.0,0.0,0.0
2050,SBI-4129P-C2N,0.0,0.0,0.0,0.0,0.0,0.0
2051,SBI-4129P-C2N,0.0,0.0,0.0,0.0,0.0,0.0
2052,SBI-4129P-C2N,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
t1 = webmetrics.groupby('Item Code').shift(1)

# Add suffix to column names
t1.columns += '_1M_ago'
t1

,Page Views_1M_ago,Organic Searches_1M_ago,New Users_1M_ago,Number of Sessions per User_1M_ago,Users_1M_ago,Weighted Avg Time on Page_1M_ago
0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
2049,0.0,0.0,0.0,0.0,0.0,0.0
2050,0.0,0.0,0.0,0.0,0.0,0.0
2051,0.0,0.0,0.0,0.0,0.0,0.0
2052,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
t2 = webmetrics.groupby('Item Code').shift(2)

t2.columns += '_2M_ago'
t2

,Page Views_2M_ago,Organic Searches_2M_ago,New Users_2M_ago,Number of Sessions per User_2M_ago,Users_2M_ago,Weighted Avg Time on Page_2M_ago
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...
2049,0.0,0.0,0.0,0.0,0.0,0.0
2050,0.0,0.0,0.0,0.0,0.0,0.0
2051,0.0,0.0,0.0,0.0,0.0,0.0
2052,0.0,0.0,0.0,0.0,0.0,0.0


Now to join all the tables together to create the final dataset.

In [ ]:
final_data = pd.concat([joined_data, t1, t2], axis = 1)

final_data

,Date,Item Code,Qty Sold,Page,bom_type ID,BOM Type,Page Views,Organic Searches,New Users,Number of Sessions per User,Users,Weighted Avg Time on Page,Page Views_1M_ago,Organic Searches_1M_ago,New Users_1M_ago,Number of Sessions per User_1M_ago,Users_1M_ago,Weighted Avg Time on Page_1M_ago,Page Views_2M_ago,Organic Searches_2M_ago,New Users_2M_ago,Number of Sessions per User_2M_ago,Users_2M_ago,Weighted Avg Time on Page_2M_ago
0,2019-07-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-08-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-09-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-10-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-11-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2049,2021-04-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2050,2021-05-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2051,2021-06-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2052,2021-07-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Rows with missing values `NaN`'s, which is due to lack of website metrics at *t-1* and *t-2* will be dropped.

In [ ]:
final_data.isna().sum()

Date                                    0
Item Code                               0
Qty Sold                                0
Page                                    0
bom_type ID                             0
BOM Type                                0
Page Views                              0
Organic Searches                        0
New Users                               0
Number of Sessions per User             0
Users                                   0
Weighted Avg Time on Page               0
Page Views_1M_ago                      79
Organic Searches_1M_ago                79
New Users_1M_ago                       79
Number of Sessions per User_1M_ago     79
Users_1M_ago                           79
Weighted Avg Time on Page_1M_ago       79
Page Views_2M_ago                     158
Organic Searches_2M_ago               158
New Users_2M_ago                      158
Number of Sessions per User_2M_ago    158
Users_2M_ago                          158
Weighted Avg Time on Page_2M_ago  

In [ ]:
final_data = final_data.dropna(axis = 0).reset_index().drop('index', axis = 1)

,Date,Item Code,Qty Sold,Page,bom_type ID,BOM Type,Page Views,Organic Searches,New Users,Number of Sessions per User,Users,Weighted Avg Time on Page,Page Views_1M_ago,Organic Searches_1M_ago,New Users_1M_ago,Number of Sessions per User_1M_ago,Users_1M_ago,Weighted Avg Time on Page_1M_ago,Page Views_2M_ago,Organic Searches_2M_ago,New Users_2M_ago,Number of Sessions per User_2M_ago,Users_2M_ago,Weighted Avg Time on Page_2M_ago
0,2019-09-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,1.0,1.0,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-10-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-11-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,1.0,1.0,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
3,2019-12-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,1.0,1.0,1.0,1.0,1.0,216.000000,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,7.0,1.0,2.0,2.0,5.0,153.285714,1.0,1.0,1.0,1.0,1.0,216.0,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1891,2021-04-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1892,2021-05-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1893,2021-06-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1894,2021-07-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


As shown above, the data shape is 1896 x 30, after we drop the 158 rows that contains NaNs.

Finally, we add one column to the data frame for each website metric that is equal to the total of the current and previous two months. 

In [ ]:
final_data["Page Views_Current_and_Past_2_Months"] = final_data["Page Views"]+final_data["Page Views_1M_ago"]+final_data["Page Views_2M_ago"]
final_data["Organic Searches_Current_and_Past_2_Months"] = final_data["Organic Searches"]+final_data["Organic Searches_1M_ago"]+final_data["Organic Searches_2M_ago"]
final_data["New Users_Current_and_Past_2_Months"] = final_data["New Users"]+final_data["New Users_1M_ago"]+final_data["New Users_2M_ago"]
final_data["Number of Sessions per User_Current_and_Past_2_Months"] = final_data["Number of Sessions per User"]+final_data["Number of Sessions per User_1M_ago"]+final_data["Number of Sessions per User_2M_ago"]
final_data["Users_Current_and_Past_2_Months"] = final_data["Users"]+final_data["Users_1M_ago"]+final_data["Users_2M_ago"]
final_data["Weighted Avg Time on Page_Current_and_Past_2_Months"] = final_data["Weighted Avg Time on Page"]+final_data["Weighted Avg Time on Page_1M_ago"]+final_data["Weighted Avg Time on Page_2M_ago"]

final_data

,Date,Item Code,Qty Sold,Page,bom_type ID,BOM Type,Page Views,Organic Searches,New Users,Number of Sessions per User,Users,Weighted Avg Time on Page,Page Views_1M_ago,Organic Searches_1M_ago,New Users_1M_ago,Number of Sessions per User_1M_ago,Users_1M_ago,Weighted Avg Time on Page_1M_ago,Page Views_2M_ago,Organic Searches_2M_ago,New Users_2M_ago,Number of Sessions per User_2M_ago,Users_2M_ago,Weighted Avg Time on Page_2M_ago,Page Views_Current_and_Past_2_Months,Organic Searches_Current_and_Past_2_Months,New Users_Current_and_Past_2_Months,Number of Sessions per User_Current_and_Past_2_Months,Users_Current_and_Past_2_Months,Weighted Avg Time on Page_Current_and_Past_2_Months
0,2019-09-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,1.0,1.0,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.000000
1,2019-10-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.000000
2,2019-11-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,1.0,1.0,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,2.0,2.0,2.0,2.0,0.000000
3,2019-12-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,1.0,1.0,1.0,1.0,1.0,216.000000,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,216.000000
4,2020-01-01,1013S-MTR,0.0,/systems/AS-1013S-MTR,2,BB,7.0,1.0,2.0,2.0,5.0,153.285714,1.0,1.0,1.0,1.0,1.0,216.0,1.0,1.0,1.0,1.0,1.0,0.0,9.0,3.0,4.0,4.0,7.0,369.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1891,2021-04-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1892,2021-05-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1893,2021-06-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1894,2021-07-01,SBI-4129P-C2N,0.0,/systems/SBI-4129P-C2N,5,Node,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


The final data shape is 1896 x 30

In [ ]:
final_data.to_csv("webdata_qtySold_byMonth.csv")
files.download('webdata_qtySold_byMonth.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# OLD JOIN

In [ ]:
# # Merge qtySold_bom_result with website data
# webdata_qtySold_byMonth = pd.merge(qtySold_bom_result, data_byMonth, how = "outer", on = ['Page', 'Date'])
# webdata_qtySold_byMonth = webdata_qtySold_byMonth.dropna(subset=['Organic Searches', 'Item Code'])
# # Advised to categorize items as "Other" if category is not specified
# webdata_qtySold_byMonth['bom_type ID'] = webdata_qtySold_byMonth['bom_type ID'].replace(np.nan, 'Other')
# webdata_qtySold_byMonth['BOM Type'] = webdata_qtySold_byMonth['BOM Type'].replace(np.nan, 'Other')
# # Advised to ignore Pages that are missing or have "/searchGlobal?q="
# index = 0
# for page in webdata_qtySold_byMonth['Page']:
#     x = re.search('^/searchGlobal', page)
#     if x != None:
#       webdata_qtySold_byMonth = webdata_qtySold_byMonth.drop([index])
    
#     index += 1
# webdata_qtySold_byMonth = webdata_qtySold_byMonth.drop(columns=['Avg. Time on Page', 'avg*sum'])
# for page in webdata_qtySold_byMonth['Page']:
#   for item in webdata_qtySold_byMonth['Item Code']:
#     x = re.search(item + '$', page)
#     if x != None:
#       webdata_qtySold_byMonth.loc[webdata_qtySold_byMonth['Page'] == page, 'Item Code'] = item
# webdata_qtySold_byMonth
# # pd.date_range('2014-10-10','2016-01-07', 
# #               freq='MS').strftime("%Y-%b").tolist()

# webdata_qtySold_byMonth['Date'].min()

In [ ]:
# webdata_qtySold_byMonth['Item Code'].fillna("empty", inplace = True)
# webdata_qtySold_byMonth = webdata_qtySold_byMonth[~webdata_qtySold_byMonth['Item Code'].str.contains('empty')]
# webdata_qtySold_byMonth['Qty Sold'] = webdata_qtySold_byMonth['Qty Sold'].fillna(0)
# webdata_qtySold_byMonth = webdata_qtySold_byMonth.fillna(0)
# webdata_qtySold_byMonth = webdata_qtySold_byMonth.sort_values(by=['Item Code', 'Date'])
# webdata_qtySold_byMonth = webdata_qtySold_byMonth.drop_duplicates(subset=['Date', 'Item Code', 'Page', 'Qty Sold'])
# webdata_qtySold_byMonth = webdata_qtySold_byMonth.reset_index()
# webdata_qtySold_byMonth.to_csv("webdata_qtySold_byMonth.csv")